In [ ]:
from sympy import *
from sympy import latex
from sympy.plotting import plot3d
from sympy.plotting.plot import plot_contour
from scipy.integrate import quad, nquad, dblquad
from sympy import init_session
%matplotlib inline
init_printing(use_unicode=True)

In [ ]:
x, y, z, zx, zxp, pxdc, pxdcp, beta, dbeta, mu, gamma = symbols('x y z \zeta_x \zeta\'_x \phi_{xdc} \phi_{xdc}\' B dB \mu \gamma ')
eta, etap, pdc, pdcp, z, zp = var(' \eta \eta\' \phi_{dc} \phi_{dc}\' \zeta \zeta\' ', real=True)

vphi = (-z+zp)/2
vphip = (z+zp)/2
#dimensionless potential
U_L = 1/(2*(1-mu)) * (z-zx)**2 + 1/(2*(1+mu)) * (zp-zxp)**2 + gamma* ((pdc-pxdc)**2 + (pdcp-pxdcp)**2)

F_U = 2 * ( -beta * cos(x) * cos(y/2) + dbeta * sin(x) * sin(y/2))

U = U_L + F_U.subs([[x,vphi],[y,pdc]]) + F_U.subs([[x,vphip],[y,pdcp]])

print('U=')
U


## Execute the two cells above and then the one below for an interactive plot of the potential

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interactive
from mpl_toolkits.mplot3d import Axes3D

# use the function keyword arguments below to set default values
def f(zxv=0, zxpv=0, pdcv=0, pdcpv=0, pxdcv=0, pxdcpv=0, mv=.5, gv=24, bv=12, dbv=0):
    params = [[zx, zxv], [zxp, zxpv], [pdc, pdcv], [pdcp, pdcpv], [pxdc,pxdcv], [pxdcp, pxdcpv], [mu,mv], [gamma,gv], [beta,bv], [dbeta,dbv]]
    
    U_sub = U.subs(params).simplify()
    
    #SET THE PLOT WINDOW HERE ###
    xmin, xmax, ymin, ymax = -6, 6, -6, 6
    resolution = 100
    #SET THE PLOT WINDOW HERE ###
    
    X,Y = np.meshgrid( np.linspace(xmin, xmax, resolution), np.linspace(ymin, ymax, resolution))
    ft = lambdify((z,zp), U_sub, 'numpy')
    U_num = ft(X,Y)
    
    fig = plt.figure(figsize=(10,10))
    ax2 = fig.add_subplot(222, projection="3d")
    ax2.plot_wireframe(X,Y,U_num)
    ax3 = fig.add_subplot(221)
    ax3.contourf(X,Y,U_num)
    for item in [ax2, ax3]:
        item.set_xlabel('$\zeta$')
        item.set_ylabel('$\zeta\'$')

    
    #print('left min:',np.min(U_num[:,:int(resolution/2)]),'right_min:',np.min(U_num[:,int(resolution/2):]))

#SET PARAMETER RANGES HERE ### 
xrng = (-6.5, 6.5, .01)
mrng = (0,2,.01)
brng = (0,30,.1)
#SET PARAMETER RANGES HERE ### 


interactive_plot = interactive(f, zxv=xrng, zxpv=xrng, pdcv=xrng, pdcpv=xrng, pxdcv=xrng, pxdcpv=xrng, mv=mrng, gv=brng, bv=brng, dbv=brng)
#output = interactive_plot.children[-1]
#output.layout.height = '900px'
interactive_plot

# Below is a model for how to plot the force vector field, saved for future use possibly. Not currently functional at the same time as the one above due to reused variable definitions

In [ ]:


x, y, z, etax, etaxp, pdcx, pdcxp, I_p, I_m, mu, L, phi_0, gamma = symbols('x y z \eta_x \eta_x\' \phi_x \phi_x\' I_+ I_- \mu L \Phi_0 \gamma ')
eta, etap, pdc, pdcp = var(' \eta \eta\' \phi_{dc} \phi_{dc}\' ', real=True)

#scaling = 1
scaling = 2 * pi / phi_0
zeta = eta * scaling
zetap = etap * scaling
vpdc = pdc * scaling
vpdcp = pdcp * scaling

phi = (-zeta+zetap)/2
phip = (zeta+zetap)/2

F = I_p * sin(x) * cos(y/2) + I_m * cos(x) * sin(y/2)

F_eta = 1/(L*(1-mu))*(etax-eta) + F.subs([[x,phi], [y,vpdc]]) - F.subs([[x,phip], [y,vpdcp]])
F_etap = 1/(L*(1+mu)) *(etaxp-etap) -F.subs([[x,phi], [y,vpdc]]) - F.subs([[x,phip], [y,vpdcp]])
F_pdc = 2*gamma/L * (pdcx-pdc) - F.subs([[x,vpdc/2],[y,2*phi]])
F_pdcp = 2*gamma/L *(pdcp- pdcxp) -  F.subs([[x,vpdcp/2],[y,2*phip]])

In [ ]:
#vector plot of the forces
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interactive
from mpl_toolkits.mplot3d import Axes3D

# use the function keyword arguments below to set default values
def f(etaxv=0, etaxpv=0, vpdcv=0, vpdcpv=0, Lv=1, Mv=.2, I_pv=1, I_mv=1):
    params = [[etax,etaxv],[etaxp,etaxpv],[vpdc,vpdcv], [vpdcp,vpdcpv], [L,Lv], [mu,Mv], [I_p,I_pv], [I_m,I_mv]]
    
    F_eta_sub = F_eta.subs(params).simplify()
    F_etap_sub = F_etap.subs(params).simplify()
    
    #SET THE PLOT WINDOW HERE ###
    pmin, pmax, dcmin, dcmax = -30, 30, -30, 30
    resolution = 20
    #SET THE PLOT WINDOW HERE ###
    
    X,Y = np.meshgrid( np.linspace(pmin, pmax, resolution), np.linspace(dcmin, dcmax, resolution))
    
    f_e = lambdify((eta,etap), F_eta_sub, 'numpy')
    f_ep = lambdify((eta,etap), F_etap_sub, 'numpy')
    
    F_eta_num = f_e(X,Y)
    F_etap_num = f_ep(X,Y) 
    
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    C = np.hypot(F_eta_num, F_etap_num)
    ax.quiver(X, Y, F_eta_num, F_etap_num, C)
    
    #print('left min:',np.min(U_num[:,:int(resolution/2)]),'right_min:',np.min(U_num[:,int(resolution/2):]))

#SET PARAMETER RANGES HERE ### 
xrng = (-6.5, 6.5, .01)
erng = (0.01,.5,.01)
brng = (0,10,.01)
Lrng = (.001,5)
#SET PARAMETER RANGES HERE ### 


interactive_plot = interactive(f, etaxv=xrng, etaxpv=xrng, vpdcv=xrng, vpdcpv=xrng, Lv=Lrng, Mv=Lrng, I_1v=brng, I_2v=brng)
#output = interactive_plot.children[-1]
#output.layout.height = '900px'
interactive_plot